In [1]:
from pyspark.sql import SparkSession


In [2]:
spark = (
    SparkSession
    .builder
    .master("local[1]")
    .appName("spark_SQL")
    .getOrCreate()
)

25/05/08 14:38:46 WARN Utils: Your hostname, DESKTOP-3F8M06L resolves to a loopback address: 127.0.1.1; using 192.168.0.8 instead (on interface eth0)
25/05/08 14:38:46 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/08 14:38:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = (
    spark
    .read
    .option('delimiter', ';')
    .option('header', 'true')
    .option('inferSchema', 'true')
    .option('enconding', 'ISO-8859-1')
    .csv('./dados/precos-gasolina-etanol-03.csv')
)

In [4]:
df.printSchema()

root
 |-- Regiao - Sigla: string (nullable = true)
 |-- Estado - Sigla: string (nullable = true)
 |-- Municipio: string (nullable = true)
 |-- Revenda: string (nullable = true)
 |-- CNPJ da Revenda: string (nullable = true)
 |-- Nome da Rua: string (nullable = true)
 |-- Numero Rua: string (nullable = true)
 |-- Complemento: string (nullable = true)
 |-- Bairro: string (nullable = true)
 |-- Cep: string (nullable = true)
 |-- Produto: string (nullable = true)
 |-- Data da Coleta: string (nullable = true)
 |-- Valor de Venda: string (nullable = true)
 |-- Valor de Compra: string (nullable = true)
 |-- Unidade de Medida: string (nullable = true)
 |-- Bandeira: string (nullable = true)



In [8]:
df.createOrReplaceTempView('combustiveis')

In [18]:
spark.sql("""
    select `Estado - Sigla`, `Produto`, `Valor de Venda`,`Valor de Compra`, `Unidade de Medida` from combustiveis
""").show(10)

+--------------+------------------+--------------+---------------+-----------------+
|Estado - Sigla|           Produto|Valor de Venda|Valor de Compra|Unidade de Medida|
+--------------+------------------+--------------+---------------+-----------------+
|            SP|          GASOLINA|          5,65|           NULL|       R$ / litro|
|            SP|            ETANOL|          3,78|           NULL|       R$ / litro|
|            AL|          GASOLINA|          6,49|           NULL|       R$ / litro|
|            AL|            ETANOL|          4,99|           NULL|       R$ / litro|
|            AL|          GASOLINA|          6,49|           NULL|       R$ / litro|
|            AL|GASOLINA ADITIVADA|          6,59|           NULL|       R$ / litro|
|            AL|            ETANOL|          4,98|           NULL|       R$ / litro|
|            AL|          GASOLINA|          6,29|           NULL|       R$ / litro|
|            AL|            ETANOL|          4,79|           NULL

In [21]:
spark.sql("""
    select * from combustiveis where `Valor de Compra` IS NOT NULL
"""
         ).show()

+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
|Regiao - Sigla|Estado - Sigla|Municipio|Revenda|CNPJ da Revenda|Nome da Rua|Numero Rua|Complemento|Bairro|Cep|Produto|Data da Coleta|Valor de Venda|Valor de Compra|Unidade de Medida|Bandeira|
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+
+--------------+--------------+---------+-------+---------------+-----------+----------+-----------+------+---+-------+--------------+--------------+---------------+-----------------+--------+



In [24]:
view_precos = (
    spark.sql("""
            select 
            `Estado - Sigla`, 
            `Produto`, 
             regexp_replace(`Valor de Venda` , ",", ".") as `Valor de Venda`,            
            `Unidade de Medida` 
        from combustiveis
    """)
)

In [30]:
view_precos.createOrReplaceTempView('view_precos')
view_precos.show(5)

+--------------+--------+--------------+-----------------+
|Estado - Sigla| Produto|Valor de Venda|Unidade de Medida|
+--------------+--------+--------------+-----------------+
|            SP|GASOLINA|          5.65|       R$ / litro|
|            SP|  ETANOL|          3.78|       R$ / litro|
|            AL|GASOLINA|          6.49|       R$ / litro|
|            AL|  ETANOL|          4.99|       R$ / litro|
|            AL|GASOLINA|          6.49|       R$ / litro|
+--------------+--------+--------------+-----------------+
only showing top 5 rows



In [38]:
df_diferenca_precos = spark.sql("""
    select
        `Estado - Sigla`,
        `Produto`,
        `Unidade de Medida`,
        MIN(`Valor de Venda`) as menor_valor, 
        MAX(`Valor de Venda`) as maior_valor,
        MAX(`Valor de Venda`) - MIN(`Valor de Venda`) as diferenca 
    from view_precos
    group by all
    order by diferenca DESC
""")

In [39]:
df_diferenca_precos.show(5)

[Stage 12:>                                                         (0 + 1) / 1]

+--------------+------------------+-----------------+-----------+-----------+------------------+
|Estado - Sigla|           Produto|Unidade de Medida|menor_valor|maior_valor|         diferenca|
+--------------+------------------+-----------------+-----------+-----------+------------------+
|            SP|GASOLINA ADITIVADA|       R$ / litro|       5.39|       8.99|3.6000000000000005|
|            SP|          GASOLINA|       R$ / litro|       5.19|       8.79|3.5999999999999988|
|            RS|            ETANOL|       R$ / litro|       3.49|       6.28|              2.79|
|            SP|            ETANOL|       R$ / litro|       3.25|       5.99|              2.74|
|            PE|            ETANOL|       R$ / litro|       3.99|       6.49|               2.5|
+--------------+------------------+-----------------+-----------+-----------+------------------+
only showing top 5 rows

